In [39]:
import pandas as pd
import folium
import folium.plugins as plugins
from folium.plugins import S
import numpy as np
from math import radians, cos, sin, sqrt, atan2

ImportError: cannot import name 'ScalerBar'

In [15]:
df = pd.read_csv('data/OMA/TDD/02_intermediate/processed_oss_tdd.csv', sep='|')
sites = pd.read_csv('data/OMA/TDD/01_raw/df_sites.csv', sep='|')
sites

,site_id,latitude,longitude,commune,ville,province,region,site_name,sector,cell_name,cell_id,cell_lac,cell_tech,azimuth,horizantal_beam_width,vertical_beam_width,downtilt,cell_band
0,AFO001,32.20720,-6.54033,Afourar,Afourar,Azilal,Béni Mellal-Khénifra,AFO001,R,AFO001R,30016.0,50700.0,3G,60.0,65.0,7.0,2.0,U2100-F3
1,AFO001,32.20720,-6.54033,Afourar,Afourar,Azilal,Béni Mellal-Khénifra,AFO001,S,AFO001S,30017.0,50700.0,3G,180.0,65.0,7.0,2.0,U2100-F3
2,AFO001,32.20720,-6.54033,Afourar,Afourar,Azilal,Béni Mellal-Khénifra,AFO001,T,AFO001T,30018.0,50700.0,3G,300.0,65.0,7.0,2.0,U2100-F3
3,AFO001,32.20720,-6.54033,Afourar,Afourar,Azilal,Béni Mellal-Khénifra,AFO001,U,AFO001U,30010.0,50700.0,3G,60.0,65.0,7.0,2.0,U2100-F1
4,AFO001,32.20720,-6.54033,Afourar,Afourar,Azilal,Béni Mellal-Khénifra,AFO001,V,AFO001V,30011.0,50700.0,3G,180.0,65.0,7.0,2.0,U2100-F1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89750,ZAI126,34.94875,-2.74344,NaN,NaN,NaN,NaN,ZAI126,NaN,ZAI126_L2,25632258.0,NaN,NaN,120.0,NaN,NaN,NaN,L1800
89751,ZAI126,34.94875,-2.74344,NaN,NaN,NaN,NaN,ZAI126,NaN,ZAI126_L3,25632259.0,NaN,NaN,200.0,NaN,NaN,NaN,L1800
89752,ZAI126,34.94875,-2.74344,NaN,NaN,NaN,NaN,ZAI126,NaN,ZAI126_L7,25632263.0,NaN,NaN,60.0,NaN,NaN,NaN,L800
89753,ZAI126,34.94875,-2.74344,NaN,NaN,NaN,NaN,ZAI126,NaN,ZAI126_L8,25632264.0,NaN,NaN,120.0,NaN,NaN,NaN,L800


In [25]:
site_localisation = df['site_id']
site_localisation = site_localisation.drop_duplicates()
site_localisation = site_localisation.to_frame()
print(type(site_localisation))
site_localisation

<class 'pandas.core.frame.DataFrame'>


,site_id
0,CA043
52,MEK642
154,MOH600
262,MEK064
365,CAS1002
...,...
624908,AGA1095
625595,AGA072
626779,AGA1153
626841,MAR1072


In [26]:
site_localisation = site_localisation.merge(sites[['site_id','latitude','longitude']], on='site_id', how='inner')

In [28]:
site_localisation = site_localisation.drop_duplicates()
site_localisation

,site_id,latitude,longitude
0,CA043,33.568500,-7.627650
27,MEK642,33.900525,-5.582839
54,MOH600,33.716300,-7.343750
81,MEK064,33.880600,-5.565680
108,CAS1002,33.473900,-7.609900
...,...,...,...
21420,AGA1095,30.399194,-9.550614
21438,AGA072,30.350392,-9.534279
21465,AGA1153,30.395978,-9.549614
21480,MAR1072,31.624778,-8.058861


In [41]:
map_center = [site_localisation['latitude'].mean(), site_localisation['longitude'].mean()]
mymap = folium.Map(location=map_center, zoom_start=2, control_scale=True)

# 3. Ajouter des marqueurs pour chaque site sur la carte
for idx, row in site_localisation.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['site_id'],
    ).add_to(mymap)
    
#mymap.add_child(folium.ScaleBar())

# 4. Sauvegarder la carte dans un fichier HTML et l'afficher
mymap.save("map.html")

In [32]:

def haversine(lat1, lon1, lat2, lon2):
    # Convertir les coordonnées en radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    # Calculer les différences
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    # Calculer la formule haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    # Rayon de la Terre en kilomètres (environ 6371 km)
    distance = 6371 * c
    return distance

In [33]:
distances = []
num_sites = len(site_localisation)
for i in range(num_sites):
    for j in range(i + 1, num_sites):
        dist = haversine(site_localisation.iloc[i]['latitude'], site_localisation.iloc[i]['longitude'],
                         site_localisation.iloc[j]['latitude'], site_localisation.iloc[j]['longitude'])
        distances.append(dist)

# 4. Calculer la distance moyenne
average_distance = np.mean(distances)

print(f"La distance moyenne entre les sites est de {average_distance:.2f} km")

La distance moyenne entre les sites est de 207.63 km
